In [2]:
import pickle
from sklearn.externals import joblib

C:\Users\avani\AnacondaLatest\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
import azureml.core
from azureml.core import Workspace
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.model import Model
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.experiment import Experiment

In [4]:
# create workspace
ws = Workspace.create(name = "RegresionModelsWS" , 
                     subscription_id = 'f2b88417-9c48-445d-887f-f79b1cd095fb',
                     resource_group = 'MLRG',
                     create_resource_group = True,
                     location = 'eastus2')

Deploying StorageAccount with name regresiostoragef5f55c836.
Deploying KeyVault with name regresiokeyvault8c7964e7.
Deploying AppInsights with name regresioinsights0a34632e.
Deployed AppInsights with name regresioinsights0a34632e. Took 13.31 seconds.
Deployed KeyVault with name regresiokeyvault8c7964e7. Took 26.78 seconds.
Deployed StorageAccount with name regresiostoragef5f55c836. Took 48.18 seconds.
Deploying Workspace with name RegresionModelsWS.
Deployed Workspace with name RegresionModelsWS. Took 117.23 seconds.


In [5]:
# write workspace to local configuration file
ws.write_config()

In [6]:
ws = Workspace.from_config()

In [7]:
ws.get_details()

{'id': '/subscriptions/f2b88417-9c48-445d-887f-f79b1cd095fb/resourceGroups/MLRG/providers/Microsoft.MachineLearningServices/workspaces/RegresionModelsWS',
 'name': 'RegresionModelsWS',
 'location': 'eastus2',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'sku': 'Basic',
 'workspaceid': 'bed9e090-2c97-444e-9cef-a7d349abee05',
 'description': '',
 'friendlyName': 'RegresionModelsWS',
 'creationTime': '2020-02-08T18:42:18.4207867+00:00',
 'keyVault': '/subscriptions/f2b88417-9c48-445d-887f-f79b1cd095fb/resourcegroups/mlrg/providers/microsoft.keyvault/vaults/regresiokeyvault8c7964e7',
 'applicationInsights': '/subscriptions/f2b88417-9c48-445d-887f-f79b1cd095fb/resourcegroups/mlrg/providers/microsoft.insights/components/regresioinsights0a34632e',
 'identityPrincipalId': '16b6fc11-f06f-4e6d-9efd-42999ea0855f',
 'identityTenantId': '8b3dd73e-4e72-4679-b191-56da1588712b',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/f2b88417-9c48-445d-887f-f79b1cd095fb/reso

In [8]:
# create azure ml experiment
exp = Experiment(workspace = ws , name = "CarPredictionExp")

In [9]:
import datetime
# start loggin metrics
run = exp.start_logging()
run.log("Experiment start time", str(datetime.datetime.now))

In [12]:
#create a test model, register in workspace
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib
import numpy as np
import pandas as pd

datadf = pd.read_csv('carage_price.csv')

X_train = datadf[['Age']]
y_train = datadf['Price']

reg = LinearRegression()
reg.fit(X_train,y_train)

joblib.dump(value = reg, filename="outputs/cars_pred_model.pkl")

['outputs/cars_pred_model.pkl']

In [13]:
# test the model
loaded_model = joblib.load("outputs/cars_pred_model.pkl")
y = loaded_model.predict([[10]])
print(y)

[11206.66666667]


In [14]:
# log some data for fun
run.log("intercept", loaded_model.intercept_)

In [15]:
# end the experiment
run.log("Experiment end time", str(datetime.datetime.now))
run.complete()

In [16]:
# register the model
from azureml.core.model import Model
model = Model.register(workspace = ws , model_path="outputs/cars_pred_model.pkl", model_name="cars_pred_model")

Registering model cars_pred_model


In [17]:
# define azure ml deployment configuration
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1 , memory_gb = 1 , 
                                              tags = {'data': 'car prices'},
                                              description = 'Cars prediction model deployment')

In [18]:
# create environment confoiguration file
model_env = CondaDependencies()
model_env.add_conda_package("scikit-learn=0.21.2")

with open("model_env.yml", "w") as f :
    f.write(model_env.serialize_to_string())
    
with open("model_env.yml", "r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

- scikit-learn=0.21.2
channels:
- conda-forge



In [19]:
%%writefile score.py
import json
import numpy as np
import os 
import pickle
from sklearn.externals import joblib
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('cars_pred_model')
    model = joblib.load(model_path)
    
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    
    y_pred = model.predict(data)
    return json.dumps(y_pred.tolist())

Overwriting score.py


In [22]:
# deploy model to azure container instance
image_config = ContainerImage.image_configuration(execution_script = "score.py" , runtime = "python" , 
                                                  conda_file= "model_env.yml")

C:\Users\avani\AnacondaLatest\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
# expose web service
service = Webservice.deploy_from_model(workspace = ws , 
                                      name = "test-svc1carpricepredictor-svc",
                                      deployment_config = aciconfig,
                                      models = [model],
                                      image_config = image_config)
service.wait_for_deployment(show_output = True)

C:\Users\avani\AnacondaLatest\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: deploy_from_model has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  


Creating image
Running................................
Succeeded
Image creation operation finished for image test-svc1carpricepredictor-svc:1, operation "Succeeded"
Running.............
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [24]:
# get seb service url
print(service.scoring_uri)

http://ae965d61-80e8-4fad-b0e4-d4d65b77df4d.eastus2.azurecontainer.io/score


In [29]:
import requests
url = 'http://ae965d61-80e8-4fad-b0e4-d4d65b77df4d.eastus2.azurecontainer.io/score'
payload = '{"data": [[10]]}'
headers = {'content-type': 'application/json'}
response = requests.post(url, data=payload, headers=headers)

In [30]:
print (response.text)

"[11206.666666666668]"
